# Extract CSV from PDF

### Install libraries

In [1]:
!pip install tabula-py

    100% |████████████████████████████████| 11.7MB 4.3MB/s eta 0:00:01


In [319]:
!pip install tqdm

    100% |████████████████████████████████| 81kB 1.8MB/s ta 0:00:011


### Import libraries

In [426]:
import math
from pathlib import Path

import tabula
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm as tqdm

### Functions

In [439]:
def get_city(row, prefix='to:'):
    to = []
    to_list = []

    for item in list(row.dropna()):
        if 'unnamed' in item.lower():
            continue
        if prefix in item.lower():
            to.append(to_list)
            to_list = []
        else:
            if ',' in item:
                city, country = item.split(',')
                to_list.append(city)
                to_list.append(country)
            else:
                to_list.append(item)
    to.append(to_list)
    to_list = []
    to.pop(0)
    return to

def get_header_row(df):
    header_row = -1
    for index in range(1000):
        try:
            for item in df.iloc[index]:
                if isinstance(item, str):
                    if 'validity' in item.lower():
                        header_row = index
        except:
            pass
    return header_row


def add_columns(df, from_place, to_place, index=0):
    df['city_from'] = from_place[index][0]
    df['country_from'] = from_place[index][1]
    df['airport_from'] = from_place[index][2]
    df['city_to'] = to_place[index][0]
    df['country_to'] = to_place[index][1]
    df['airport_to'] = to_place[index][2]
    return df


def convert_df(df, from_place, to_place):
    left_df = df.iloc[:,:7]
    left_df = left_df.rename(columns = {'Trave\rTime': 'Travel\rTime'}, inplace = False)
    

    right_df = df.iloc[:,7:]
    right_df = right_df.rename(columns = {'Trave\rTim': 'Travel\rTime'}, inplace = False)

    left_df = add_columns(left_df, from_place, to_place, index=0)
    right_df = add_columns(right_df, from_place, to_place, index=1)

    frames = [left_df, right_df]
    result_df = pd.concat(frames)

    result_df = result_df.reset_index(drop=True)
    result_df = result_df.dropna()

    result_df['Travel\rTime'] = [value if value[-1] == 'M' else value + 'M' for value in result_df['Travel\rTime']]
    
    return result_df

def delete_nan_and_unnamed(df):
    new_df_list = []
    df_list = df.values.tolist()
    for row in df_list:
        new_row = []
        day_numbers = []
        for value in row:
            if isinstance(value, str):
                value = value.strip()
            if 'unnamed' in str(value).lower() or value != value:
                pass
            else:
                if isinstance(value, float) and 0 < value < 8:
                    day_numbers.append(str(int(value)))
                elif isinstance(value, int) and 0 < value < 8:
                    day_numbers.append(str(int(value)))
                elif isinstance(value, str) and value.isnumeric() and 0 < int(value) < 8:
                    day_numbers.append(str(int(value)))
                else:
                    if day_numbers:
                        new_row.append(''.join(day_numbers))
                        day_numbers = []
                    new_row.append(value)
        new_df_list.append(new_row)
    df = pd.DataFrame(new_df_list)
    return df

### Start converter

In [465]:
page_start = 1002
page_end = 2026

file = "data/Skyteam_Timetable.pdf"
result_df = None

for page in tqdm(range(page_start, page_end+1)):   
    try:
        table = tabula.read_pdf(file, pages=page)
        df = table[0]

        header_row = get_header_row(df)

        if header_row >= 0:   
            from_place = get_city(df.columns, prefix='om:')
            to_place = get_city(df.iloc[0], prefix='o:')

            df.columns = df.iloc[header_row]
            old_df_columns = df.columns
            df = df.drop(header_row)
            df = df.dropna()

            temp_df = convert_df(df, from_place, to_place)
            if result_df is not None:
                result_df = pd.concat([result_df, temp_df])
            else:
                result_df = temp_df
        else:
            df.loc[len(df)] = df.columns
                    
            df = delete_nan_and_unnamed(df)
            df = df.dropna()
        
            if len(df.columns) == 7:
                df.columns = old_df_columns[:7]
            else:
                df.columns = old_df_columns
                
            df = df.reset_index(drop=True)
    
            temp_df = convert_df(df, from_place, to_place)

            result_df = pd.concat([result_df, temp_df])
    except Exception as e:
        print(f"Exception on page {page}: {e}")



filename = 'from_pdf.csv'
file_path = Path(filename)

file_exists = file_path.exists()

result_df = result_df.rename(columns = {
    'Dep\rTime': 'Dep_Time', 
    'Arr\rTime': 'Arr_Time',
    'Travel\rTime': 'Travel_Time'
        }, 
        inplace = False)

  0%|          | 0/1024 [00:00<?, ?it/s]

Exception on page 1009: too many values to unpack (expected 2)
Exception on page 1015: list index out of range
Exception on page 1016: list index out of range
Exception on page 1017: list index out of range
Exception on page 1018: list index out of range
Exception on page 1053: Length mismatch: Expected axis has 12 elements, new values have 14 elements
Exception on page 1056: Length mismatch: Expected axis has 10 elements, new values have 14 elements
Exception on page 1057: Length mismatch: Expected axis has 12 elements, new values have 14 elements
Exception on page 1058: Length mismatch: Expected axis has 12 elements, new values have 14 elements
Exception on page 1085: Length mismatch: Expected axis has 8 elements, new values have 14 elements
Exception on page 1109: too many values to unpack (expected 2)
Exception on page 1127: too many values to unpack (expected 2)
Exception on page 1129: Plan shapes are not aligned
Exception on page 1130: Plan shapes are not aligned
Exception on pag

In [471]:
result_df

1,Validity,Days,Dep_Time,Arr_Time,Flight,Aircraft,Travel_Time,city_from,country_from,airport_from,city_to,country_to,airport_to
0,01 Jan - 29 Jan,1 3 6,14:30,19:15,KL759,789,11H45M,Amsterdam,Netherlands,AMS,San Jose,Costa Rica,SJ
1,03 Jan - 31 Jan,5,14:30,19:10,KL759,789,11H40M,Amsterdam,Netherlands,AMS,San Jose,Costa Rica,SJ
2,20 Dec - 27 Dec,5,15:10,19:45,KL759,789,11H35M,Amsterdam,Netherlands,AMS,San Jose,Costa Rica,SJ
3,02 Nov - 30 Dec,1 3 6,15:20,20:05,KL759,789,11H45M,Amsterdam,Netherlands,AMS,San Jose,Costa Rica,SJ
4,01 Jan - 29 Jan,1 3 6,14:30,19:15,KL759,789,11H45M,Amsterdam,Netherlands,AMS,San Jose,Costa Rica,SJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,16 Nov - 21 Nov,1 34 6,18:35,21:43,DL2445,717,2H8M,Atlanta GA,United States of,AT,Dallas TX,United States of,DA
31,24 Nov - 26 Nov,12 7,18:35,21:43,DL2445,717,2H8M,Atlanta GA,United States of,AT,Dallas TX,United States of,DA
32,09 Nov - 09 Nov,6,18:37,21:45,DL2445,717,2H8M,Atlanta GA,United States of,AT,Dallas TX,United States of,DA
33,27 Nov - 21 Dec,123456,18:42,21:45,DL2445,717,2H3M,Atlanta GA,United States of,AT,Dallas TX,United States of,DA


### Rename columns

In [468]:
result_df = result_df.rename(columns = {
    'Dep\rTime': 'Dep_Time', 
    'Arr\rTime': 'Arr_Time',
    'Travel\rTime': 'Travel_Time'
        }, 
        inplace = False)

result_df.columns

Index(['Validity', 'Days', 'Dep_Time', 'Arr_Time', 'Flight', 'Aircraft',
       'Travel_Time', 'city_from', 'country_from', 'airport_from', 'city_to',
       'country_to', 'airport_to'],
      dtype='object', name=1)

### Save dataset and check

In [469]:
result_df.to_csv('out.csv', index=False, sep=';')

In [470]:
df_full = pd.read_csv('out.csv', sep=';')
df_full

,Validity,Days,Dep_Time,Arr_Time,Flight,Aircraft,Travel_Time,city_from,country_from,airport_from,city_to,country_to,airport_to
0,01 Jan - 29 Jan,1 3 6,14:30,19:15,KL759,789,11H45M,Amsterdam,Netherlands,AMS,San Jose,Costa Rica,SJ
1,03 Jan - 31 Jan,5,14:30,19:10,KL759,789,11H40M,Amsterdam,Netherlands,AMS,San Jose,Costa Rica,SJ
2,20 Dec - 27 Dec,5,15:10,19:45,KL759,789,11H35M,Amsterdam,Netherlands,AMS,San Jose,Costa Rica,SJ
3,02 Nov - 30 Dec,1 3 6,15:20,20:05,KL759,789,11H45M,Amsterdam,Netherlands,AMS,San Jose,Costa Rica,SJ
4,01 Jan - 29 Jan,1 3 6,14:30,19:15,KL759,789,11H45M,Amsterdam,Netherlands,AMS,San Jose,Costa Rica,SJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71667,16 Nov - 21 Nov,1 34 6,18:35,21:43,DL2445,717.0,2H8M,Atlanta GA,United States of,AT,Dallas TX,United States of,DA
71668,24 Nov - 26 Nov,12 7,18:35,21:43,DL2445,717.0,2H8M,Atlanta GA,United States of,AT,Dallas TX,United States of,DA
71669,09 Nov - 09 Nov,6,18:37,21:45,DL2445,717.0,2H8M,Atlanta GA,United States of,AT,Dallas TX,United States of,DA
71670,27 Nov - 21 Dec,123456,18:42,21:45,DL2445,717.0,2H3M,Atlanta GA,United States of,AT,Dallas TX,United States of,DA
